# Expected Loss From Accepting Defaulters vs Rejecting Non-Defaulters

Throughout this project we have frequently come across the concept of risking losing a significant proportion of the principal when granting loans to those who will default, and comparing that expected loss with that incurred by rejecting a borrower who would've fully paid, thus losing their interest. Here, we will analytically quantify the expected loss resulting from accepting a defaulting borrower vs rejecting a full-paying borrower.


In [ ]:
use total payment from df